# Chapter 55: Human-in-the-Loop AI Systems
Confidence thresholds, escalation routing, approval workflows.


In [ ]:
import random
print("Human-in-the-Loop ready")


In [ ]:
def route(conf,high_impact=False):
    if high_impact and conf<0.95: return "escalate"
    if conf>=0.90: return "auto_approve"
    if conf>=0.70: return "soft_approve"
    return "escalate"
print("Conf | High Impact | Decision")
print("-"*35)
for conf,hi in [(0.97,False),(0.85,False),(0.60,False),(0.92,True),(0.99,False),(0.75,True)]:
    print(str(conf)+" | "+str(hi)+" | "+route(conf,hi))


In [ ]:
class ApprovalWF:
    def __init__(self): self.pending={}; self.done=[]; self.n=1
    def submit(self,action,conf): tid="APPR-"+str(self.n).zfill(4); self.n+=1; self.pending[tid]={"id":tid,"action":action,"conf":conf}; return tid
    def approve(self,tid,by="soc-1"):
        if tid in self.pending: t=self.pending.pop(tid); t["by"]=by; self.done.append(t); return t
wf=ApprovalWF()
t1=wf.submit("Quarantine 10.0.1.55",0.75); t3=wf.submit("Deploy emergency patch",0.89)
wf.approve(t1); wf.approve(t3,"security-lead")
print("Pending:",len(wf.pending),"| Approved:",len(wf.done))
for t in wf.done: print(" ["+t["id"]+"]",t["action"][:40],"by",t["by"])
random.seed(42)
preds=[(round(random.uniform(0.5,1.0),2),random.random()<random.uniform(0.5,1.0)) for _ in range(200)]
buckets={t:[] for t in [0.5,0.6,0.7,0.8,0.9]}
for conf,correct in preds:
    for thr in sorted(buckets.keys(),reverse=True):
        if conf>=thr: buckets[thr].append(correct); break
for thr,results in sorted(buckets.items()):
    if results:
        acc=sum(results)/len(results)
        cal="Good" if abs(acc-thr)<0.1 else "Over" if acc<thr else "Under"
        print(str(thr)+" | "+str(len(results))+" | "+str(round(acc*100,1))+"% | "+cal)
